# Complete AI Agent System with OpenAI AgentKit
==============================================

This example demonstrates a production-ready customer service agent that combines:
1. Basic agent setup with proper configuration
2. Multi-step workflow execution
3. Tool/function integration
4. Stateful conversation with memory
5. Evaluation and performance monitoring

Prerequisites:

```bash
    pip install openai python-dotenv
```

Set your [OpenAI API Key](https://platform.openai.com/settings/organization/api-keys): 

```bash
    OPENAI_API_KEY='your-api-key-here
```

In [ ]:
import os
import json
from datetime import datetime
from typing import List, Dict, Optional, Any
from openai import OpenAI
from dataclasses import dataclass, field
import logging
from dotenv import load_dotenv, find_dotenv
import warnings

# Load environment variables from .env file
load_dotenv(find_dotenv())

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
logging.basicConfig(level=logging.INFO)

In [ ]:
# Configure logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

In [ ]:
# dataclass to store conversation metadata
@dataclass
class ConversationMetadata:
    """Store metadata about the conversation"""

    created_at: str = field(default_factory=lambda: datetime.now().isoformat())
    total_interactions: int = 0
    total_tool_calls: int = 0
    last_interaction: Optional[str] = None
    customer_id: Optional[str] = None
    sentiment_scores: List[float] = field(default_factory=list)


@dataclass
class EvaluationResult:
    """Store evaluation results"""

    timestamp: str
    interaction_id: int
    user_input: str
    agent_response: str
    score: float
    reasoning: str
    tool_used: Optional[str] = None

In [ ]:
class CustomerServiceAgent:
    """
    A comprehensive customer service agent that demonstrates all AgentKit capabilities:
    - Basic setup and configuration
    - Multi-step workflows
    - Tool integration (order lookup, refund processing, etc.)
    - Stateful conversations with memory
    - Performance evaluation
    """

    def __init__(self, model: str = "gpt-5-2025-08-07", enable_evaluation: bool = True):
        """
        Initialize the customer service agent

        Args:
            model: OpenAI model to use
            enable_evaluation: Whether to enable automatic evaluation
        """
        # Safe OpenAI client initialization: try to create a client, but allow demo fallback
        self.client = None
        self.client_available = False
        self.last_client_error: Optional[str] = None
        try:
            api_key = os.getenv("OPENAI_API_KEY")
            if not api_key:
                raise ValueError("OPENAI_API_KEY not found in environment")
            self.client = OpenAI(api_key=api_key)
            self.client_available = True
        except Exception as e:
            logger.warning(
                f"OpenAI client unavailable - entering demo fallback mode: {str(e)}"
            )
            self.client = None
            self.client_available = False
            self.last_client_error = str(e)

        self.model = model
        self.enable_evaluation = enable_evaluation

        # Define the agent's system prompt
        self.system_prompt = """You are an intelligent customer service agent for TechStore, 
        an online electronics retailer. Your responsibilities include:
        
        1. Answering customer inquiries professionally and empathetically
        2. Looking up order information when needed
        3. Processing refunds and returns
        4. Checking product inventory
        5. Escalating complex issues to human agents
        
        Always be polite, helpful, and ask for necessary information (like order numbers) 
        when needed. Use the available tools to provide accurate information."""

        # Initialize conversation history with system prompt
        self.conversation_history: List[Dict[str, str]] = [
            {"role": "system", "content": self.system_prompt}
        ]

        # Initialize metadata tracking
        self.metadata = ConversationMetadata()

        # Initialize evaluation results
        self.evaluation_results: List[EvaluationResult] = []

        # Define available tools/functions
        self.tools = self._define_tools()

        # Map function names to implementations
        self.available_functions = {
            "lookup_order": self._lookup_order,
            "process_refund": self._process_refund,
            "check_inventory": self._check_inventory,
            "escalate_to_human": self._escalate_to_human,
            "analyze_sentiment": self._analyze_sentiment,
        }

        logger.info(f"CustomerServiceAgent initialized with model: {model}")

    def _define_tools(self) -> List[Dict]:
        """Define the tools/functions available to the agent"""
        return [
            {
                "type": "function",
                "function": {
                    "name": "lookup_order",
                    "description": "Look up order details by order number",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "order_number": {
                                "type": "string",
                                "description": "The order number (e.g., ORD-12345)",
                            }
                        },
                        "required": ["order_number"],
                    },
                },
            },
            {
                "type": "function",
                "function": {
                    "name": "process_refund",
                    "description": "Process a refund for an order",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "order_number": {
                                "type": "string",
                                "description": "The order number to refund",
                            },
                            "reason": {
                                "type": "string",
                                "description": "Reason for the refund",
                            },
                            "amount": {
                                "type": "number",
                                "description": "Refund amount in USD",
                            },
                        },
                        "required": ["order_number", "reason"],
                    },
                },
            },
            {
                "type": "function",
                "function": {
                    "name": "check_inventory",
                    "description": "Check if a product is in stock",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "product_name": {
                                "type": "string",
                                "description": "Name of the product",
                            }
                        },
                        "required": ["product_name"],
                    },
                },
            },
            {
                "type": "function",
                "function": {
                    "name": "escalate_to_human",
                    "description": "Escalate the issue to a human agent",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "issue_description": {
                                "type": "string",
                                "description": "Description of the issue to escalate",
                            },
                            "priority": {
                                "type": "string",
                                "enum": ["low", "medium", "high", "urgent"],
                                "description": "Priority level of the issue",
                            },
                        },
                        "required": ["issue_description", "priority"],
                    },
                },
            },
            {
                "type": "function",
                "function": {
                    "name": "analyze_sentiment",
                    "description": "Analyze the sentiment of the customer's message",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "message": {
                                "type": "string",
                                "description": "The customer's message to analyze",
                            }
                        },
                        "required": ["message"],
                    },
                },
            },
        ]

    # ===== TOOL IMPLEMENTATIONS (Simulated Backend Functions) =====

    def _lookup_order(self, order_number: str) -> str:
        """Simulate order lookup from database"""
        logger.info(f"Looking up order: {order_number}")

        # Simulated database
        orders_db = {
            "ORD-12345": {
                "order_number": "ORD-12345",
                "customer_name": "John Doe",
                "product": "Wireless Headphones Pro",
                "status": "Delivered",
                "order_date": "2025-09-15",
                "delivery_date": "2025-09-18",
                "amount": 129.99,
                "tracking_number": "TRK-789012",
            },
            "ORD-67890": {
                "order_number": "ORD-67890",
                "customer_name": "Jane Smith",
                "product": "Smart Watch Ultra",
                "status": "In Transit",
                "order_date": "2025-10-05",
                "estimated_delivery": "2025-10-10",
                "amount": 399.99,
                "tracking_number": "TRK-456789",
            },
        }

        order = orders_db.get(order_number)
        if order:
            return json.dumps(order, indent=2)
        else:
            return json.dumps(
                {
                    "error": "Order not found",
                    "message": f"No order found with number {order_number}",
                }
            )

    def _process_refund(
        self, order_number: str, reason: str, amount: Optional[float] = None
    ) -> str:
        """Simulate refund processing"""
        logger.info(f"Processing refund for order: {order_number}")

        # First lookup the order
        order_data = json.loads(self._lookup_order(order_number))

        if "error" in order_data:
            return json.dumps(
                {"success": False, "message": "Cannot process refund - order not found"}
            )

        # Process refund
        refund_amount = amount if amount else order_data.get("amount", 0)

        return json.dumps(
            {
                "success": True,
                "refund_id": f"REF-{datetime.now().strftime('%Y%m%d%H%M%S')}",
                "order_number": order_number,
                "refund_amount": refund_amount,
                "reason": reason,
                "status": "Approved",
                "processing_time": "3-5 business days",
                "message": "Refund has been approved and will be credited to your original payment method",
            }
        )

    def _check_inventory(self, product_name: str) -> str:
        """Simulate inventory check"""
        logger.info(f"Checking inventory for: {product_name}")

        # Simulated inventory database
        inventory = {
            "wireless headphones": {"in_stock": True, "quantity": 45, "price": 129.99},
            "smart watch": {"in_stock": True, "quantity": 12, "price": 399.99},
            "laptop": {
                "in_stock": False,
                "quantity": 0,
                "price": 1299.99,
                "restock_date": "2025-10-15",
            },
            "phone case": {"in_stock": True, "quantity": 250, "price": 29.99},
        }

        product_key = product_name.lower()
        for key in inventory.keys():
            if key in product_key or product_key in key:
                return json.dumps(inventory[key])

        return json.dumps(
            {"in_stock": False, "message": "Product not found in our inventory"}
        )

    def _escalate_to_human(self, issue_description: str, priority: str) -> str:
        """Simulate escalation to human agent"""
        logger.info(f"Escalating issue with priority: {priority}")

        ticket_id = f"TICKET-{datetime.now().strftime('%Y%m%d%H%M%S')}"

        return json.dumps(
            {
                "escalated": True,
                "ticket_id": ticket_id,
                "priority": priority,
                "issue": issue_description,
                "estimated_response_time": (
                    "30 minutes" if priority in ["high", "urgent"] else "2 hours"
                ),
                "message": f"Your issue has been escalated to our support team. Ticket ID: {ticket_id}",
            }
        )

    def _api_chat(self, **kwargs):
        """Safe wrapper for OpenAI chat/completions.create.

        If the OpenAI client is not available or an API error occurs, this
        returns a lightweight simulated response object compatible with the
        rest of the code so the notebook can run offline for demo purposes.
        """
        # If client isn't available, return a demo response object
        if not self.client_available:
            class _Msg:
                def __init__(self, content):
                    self.content = content
                    self.tool_calls = []

            class _Choice:
                def __init__(self, message):
                    self.message = message

            class _Resp:
                def __init__(self, choice):
                    self.choices = [choice]

            demo_text = (
                "[DEMO MODE] OpenAI client unavailable. This is a simulated response "
                "so the notebook demonstration can continue offline."
            )
            return _Resp(_Choice(_Msg(demo_text)))

        # Otherwise, call the real API but catch errors and convert to exceptions
        try:
            return self.client.chat.completions.create(**kwargs)
        except Exception as e:
            # Log and re-raise so callers can handle as needed
            logger.error(f"OpenAI API error in _api_chat: {str(e)}")
            raise

    def _analyze_sentiment(self, message: str) -> str:
        """Analyze sentiment of customer message"""
        logger.info("Analyzing message sentiment")

        # Use GPT to analyze sentiment via safe wrapper
        try:
            response = self._api_chat(
                model="gpt-5-2025-08-07",
                messages=[
                    {
                        "role": "user",
                        "content": f"""Analyze the sentiment of this customer message and return a JSON with:
                    - sentiment: positive, neutral, or negative
                    - score: a number from -1 (very negative) to 1 (very positive)
                    - emotions: list of detected emotions
                    
                    Message: {message}
                    
                    Return only valid JSON.""",
                    }
                ],
                response_format={"type": "json_object"},
            )

            # If in demo mode, craft a neutral sentiment payload
            if not self.client_available:
                sentiment_data = {"sentiment": "neutral", "score": 0, "emotions": []}
            else:
                sentiment_data = json.loads(response.choices[0].message.content)

            # Store sentiment score
            if "score" in sentiment_data:
                self.metadata.sentiment_scores.append(sentiment_data["score"])

            return json.dumps(sentiment_data)
        except Exception as e:
            logger.error(f"Error analyzing sentiment: {str(e)}")
            return json.dumps({"sentiment": "neutral", "score": 0, "error": str(e)})

    # ===== CORE AGENT METHODS =====

    def chat(self, user_message: str, customer_id: Optional[str] = None) -> str:
        """
        Main chat method that processes user input and returns agent response

        Args:
            user_message: The user's message
            customer_id: Optional customer identifier

        Returns:
            The agent's response
        """
        try:
            logger.info(f"Processing message: {user_message[:50]}...")

            # Update metadata
            if customer_id:
                self.metadata.customer_id = customer_id

            # Add user message to conversation history
            self.conversation_history.append({"role": "user", "content": user_message})

            # Get agent response (may include tool calls)
            response = self._get_agent_response()

            # Update metadata
            self.metadata.total_interactions += 1
            self.metadata.last_interaction = datetime.now().isoformat()

            # Evaluate if enabled
            if self.enable_evaluation:
                self._evaluate_interaction(user_message, response)

            logger.info("Message processed successfully")
            return response

        except Exception as e:
            logger.error(f"Error in chat: {str(e)}")
            return (
                f"I apologize, but I encountered an error: {str(e)}. Please try again."
            )

    def _get_agent_response(self) -> str:
        """Get response from the agent, handling tool calls"""
        # Initial API call via safe wrapper
        response = self._api_chat(
            model=self.model,
            messages=self.conversation_history,
            tools=self.tools,
            tool_choice="auto",
            temperature=0.7,
        )

        response_message = response.choices[0].message
        tool_calls = getattr(response_message, "tool_calls", [])

        # If no tool calls, return the response
        if not tool_calls:
            assistant_message = getattr(response_message, "content", "")
            self.conversation_history.append(
                {"role": "assistant", "content": assistant_message}
            )
            return assistant_message

        # Handle tool calls
        self.conversation_history.append(response_message)

        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)

            logger.info(f"Calling tool: {function_name} with args: {function_args}")

            # Execute the function
            function_response = self.available_functions[function_name](**function_args)

            # Add function response to conversation
            self.conversation_history.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": function_response,
                }
            )

            self.metadata.total_tool_calls += 1

        # Get final response after tool execution
        final_response = self._api_chat(
            model=self.model, messages=self.conversation_history, temperature=0.7
        )

        final_message = final_response.choices[0].message.content
        self.conversation_history.append(
            {"role": "assistant", "content": final_message}
        )

        return final_message

    def _evaluate_interaction(self, user_input: str, agent_response: str):
        """Evaluate the quality of an interaction"""
        try:
            eval_prompt = f"""Evaluate this customer service interaction:

            Customer: {user_input}
            Agent: {agent_response}

            Rate the agent's response on a scale of 1-10 considering:
            - Helpfulness and accuracy
            - Professionalism and empathy
            - Appropriate use of tools
            - Clarity and completeness

            Return JSON with 'score' (1-10) and 'reasoning' (brief explanation).
            """

            eval_response = self._api_chat(
                model=self.model,
                messages=[{"role": "user", "content": eval_prompt}],
                response_format={"type": "json_object"},
            )

            if not self.client_available:
                eval_data = {"score": 5, "reasoning": "Demo fallback - no live evaluation"}
            else:
                eval_data = json.loads(eval_response.choices[0].message.content)

            # Store evaluation result
            result = EvaluationResult(
                timestamp=datetime.now().isoformat(),
                interaction_id=self.metadata.total_interactions,
                user_input=user_input,
                agent_response=agent_response,
                score=eval_data.get("score", 0),
                reasoning=eval_data.get("reasoning", "No reasoning provided"),
            )

            self.evaluation_results.append(result)
            logger.info(f"Evaluation score: {result.score}/10")

        except Exception as e:
            logger.error(f"Error evaluating interaction: {str(e)}")

    # ===== WORKFLOW METHODS =====

    def execute_workflow(
        self, workflow_steps: List[Dict[str, str]]
    ) -> List[Dict[str, Any]]:
        """
        Execute a multi-step workflow

        Args:
            workflow_steps: List of steps, each with 'name' and 'instruction'

        Returns:
            List of results for each step
        """
        logger.info(f"Executing workflow with {len(workflow_steps)} steps")
        results = []

        for i, step in enumerate(workflow_steps):
            logger.info(f"Step {i+1}/{len(workflow_steps)}: {step['name']}")

            response = self.chat(step["instruction"])

            results.append(
                {
                    "step_number": i + 1,
                    "step_name": step["name"],
                    "instruction": step["instruction"],
                    "response": response,
                }
            )

        return results

    # ===== REPORTING AND ANALYTICS =====

    def get_performance_report(self) -> str:
        """Generate a comprehensive performance report"""
        if not self.evaluation_results:
            return "No evaluation data available yet."

        scores = [r.score for r in self.evaluation_results]
        avg_score = sum(scores) / len(scores)

        avg_sentiment = (
            sum(self.metadata.sentiment_scores) / len(self.metadata.sentiment_scores)
            if self.metadata.sentiment_scores
            else 0
        )

        report = f"""
╔══════════════════════════════════════════════════════════════╗
║           CUSTOMER SERVICE AGENT PERFORMANCE REPORT          ║
╚══════════════════════════════════════════════════════════════╝

📊 OVERALL METRICS
─────────────────────────────────────────────────────────────
Total Interactions:        {self.metadata.total_interactions}
Total Tool Calls:          {self.metadata.total_tool_calls}
Average Response Score:    {avg_score:.2f}/10
Average Sentiment:         {avg_sentiment:.2f} ({self._sentiment_label(avg_sentiment)})
Session Duration:          {self._calculate_duration()}

📈 DETAILED EVALUATION RESULTS
─────────────────────────────────────────────────────────────
"""

        for i, result in enumerate(
            self.evaluation_results[-5:], 1
        ):  # Last 5 interactions
            report += f"""
Interaction #{result.interaction_id}
  Score:     {result.score}/10
  Time:      {result.timestamp.split('T')[1][:8]}
  Input:     {result.user_input[:60]}...
  Response:  {result.agent_response[:60]}...
  Reasoning: {result.reasoning}
"""

        report += f"""
─────────────────────────────────────────────────────────────

💡 RECOMMENDATIONS
"""

        if avg_score < 7:
            report += "  • Consider improving response quality and accuracy\n"
        if avg_sentiment < -0.3:
            report += "  • Focus on more empathetic responses\n"
        if (
            self.metadata.total_tool_calls / max(self.metadata.total_interactions, 1)
            < 0.3
        ):
            report += "  • Agent may need to use tools more effectively\n"
        if avg_score >= 8:
            report += "  • Excellent performance! Maintain current quality.\n"

        report += "\n╚══════════════════════════════════════════════════════════════╝\n"

        return report

    def _sentiment_label(self, score: float) -> str:
        """Convert sentiment score to label"""
        if score > 0.5:
            return "Very Positive"
        elif score > 0:
            return "Positive"
        elif score > -0.5:
            return "Negative"
        else:
            return "Very Negative"

    def _calculate_duration(self) -> str:
        """Calculate session duration"""
        if not self.metadata.last_interaction:
            return "N/A"

        start = datetime.fromisoformat(self.metadata.created_at)
        end = datetime.fromisoformat(self.metadata.last_interaction)
        duration = end - start

        minutes = duration.seconds // 60
        seconds = duration.seconds % 60

        return f"{minutes}m {seconds}s"

    def get_conversation_summary(self) -> str:
        """Get a summary of the conversation"""
        summary_prompt = (
            "Provide a concise summary of our conversation in 3-4 sentences."
        )
        return self.chat(summary_prompt)

    def reset_conversation(self):
        """Reset the conversation while keeping system prompt"""
        logger.info("Resetting conversation")
        self.conversation_history = [{"role": "system", "content": self.system_prompt}]
        self.metadata = ConversationMetadata()

In [ ]:
def main():
    """
    Demonstrate the complete agent system with various scenarios
    """
    print("\n" + "=" * 70)
    print("  COMPLETE AI AGENT SYSTEM - CUSTOMER SERVICE BOT DEMONSTRATION")
    print("=" * 70 + "\n")

    # Try different models in order of preference
    available_models = ["gpt-4o", "gpt-4-turbo", "gpt-3.5-turbo"]
    agent = None

    for model in available_models:
        try:
            print(f"🔄 Attempting to initialize with model: {model}")
            agent = CustomerServiceAgent(model=model, enable_evaluation=True)

            # Test the agent with a simple message
            test_response = agent.chat("Hello")
            if "error" not in test_response.lower():
                print(f"✅ Successfully initialized with model: {model}")
                break
            else:
                print(f"❌ Model {model} failed, trying next...")
                agent = None

        except Exception as e:
            print(f"❌ Model {model} failed: {str(e)}")
            continue

    if agent is None:
        print("🚫 All models failed. Please check your API key and billing.")
        return

    # Rest of your demonstration code...
    print("\n📌 SCENARIO 1: Order Status Inquiry")
    print("-" * 70)

    response1 = agent.chat(
        "Hi, I'd like to check the status of my order ORD-12345", customer_id="CUST-001"
    )
    print(f"Customer: Hi, I'd like to check the status of my order ORD-12345")
    print(f"Agent: {response1}\n")

    response2 = agent.chat(
        "I'm really upset! The headphones I ordered (ORD-12345) broke after just 2 weeks. I want a full refund immediately!"
    )
    print(
        f"Customer: I'm really upset! The headphones broke after 2 weeks. I want a refund!"
    )
    print(f"Agent: {response2}\n")

    # ===== SCENARIO 3: Inventory Check =====
    print("\n📌 SCENARIO 3: Product Availability Check")
    print("-" * 70)

    response3 = agent.chat(
        "Do you have any smart watches in stock? I'm interested in buying one."
    )
    print(f"Customer: Do you have any smart watches in stock?")
    print(f"Agent: {response3}\n")

    # ===== SCENARIO 4: Complex Issue Requiring Escalation =====
    print("\n📌 SCENARIO 4: Complex Issue Escalation")
    print("-" * 70)

    response4 = agent.chat(
        "I've been charged twice for my order and my account shows an incorrect balance. This is urgent!"
    )
    print(f"Customer: I've been charged twice! This is urgent!")
    print(f"Agent: {response4}\n")

    # ===== SCENARIO 5: Multi-Step Workflow =====
    print("\n📌 SCENARIO 5: Multi-Step Workflow - Order Investigation")
    print("-" * 70)

    workflow = [
        {"name": "Lookup Order", "instruction": "Look up order ORD-67890"},
        {
            "name": "Check Inventory",
            "instruction": "Check if we have Smart Watch Ultra in stock",
        },
        {
            "name": "Provide Update",
            "instruction": "Based on the order and inventory info, give me a complete status update",
        },
    ]

    workflow_results = agent.execute_workflow(workflow)

    for result in workflow_results:
        print(f"\n  Step {result['step_number']}: {result['step_name']}")
        print(f"  Response: {result['response'][:150]}...\n")

    # ===== GENERATE PERFORMANCE REPORT =====
    print("\n📊 PERFORMANCE REPORT")
    print("=" * 70)
    print(agent.get_performance_report())

    # ===== GET CONVERSATION SUMMARY =====
    print("\n📝 CONVERSATION SUMMARY")
    print("-" * 70)
    summary = agent.get_conversation_summary()
    print(summary)
    print("\n" + "=" * 70)
    print("  DEMONSTRATION COMPLETE")
    print("=" * 70 + "\n")

In [ ]:
if __name__ == "__main__":
    main()